In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='gsk_'
)

In [3]:
response = llm.invoke("The first person to land on Mars was...")
print(response)

content="That's not correct.  To date, no human has landed on Mars. While there have been numerous robotic missions to Mars, including rovers like NASA's Curiosity and Perseverance, as well as landers and orbiters from various countries and organizations, no human has yet visited the planet." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 44, 'total_tokens': 107, 'completion_time': 0.252, 'prompt_time': 0.010624643, 'queue_time': 0.25828095100000004, 'total_time': 0.262624643}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None} id='run-7ceb5139-ab89-4c40-bdc3-22794e505693-0' usage_metadata={'input_tokens': 44, 'output_tokens': 63, 'total_tokens': 107}


In [1]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.upwork.com/freelance-jobs/apply/Full-Stack-Developer_~021833906032048898811")

page_data = loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Just a moment...Enable JavaScript and cookies to continue'

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCITON:
    The scraped text is from the job details page of a website.
    Your job is to extract the jo postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills`, and `description`.
    Only return  the valid JSON. No tags. Fully Formatted. No escape characters.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
type(res.content)

str

In [19]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Information Technology Specialist (Direct Hire)',
 'experience': 'GS 9 - 12',
 'skills': ['Attention to Detail',
  'Customer Service',
  'Database Management Systems',
  'Information Management',
  'Problem Solving',
  'Oral Communication'],
 'description': "Establish and implement procedures and methodologies to improve the effectiveness and efficiency in the life cycle management of information such as Agency Records as it aligns with National Archives standards and policies. Develop and conduct training, presentations, consultation, and workshops on records management responsibilities including identification of Federal records in all formats and media across NASA’s platforms and systems. Responsible for the formulation, dissemination, and compliance of Agency policies, objectives, and strategic plans relating to records management solutions and technical configurations that enable those policies and objectives. Develop automation to streamline repetitive tasks, automate p

In [20]:
type(json_res)

dict

In [23]:
import pandas as pd

df = pd.read_csv("./app/resources/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import chromadb, uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [35]:
links = collection.query(
    query_texts=job['skills'],
    n_results=2
).get("metadatas")

print(links)

[[{'links': 'https://example.com/ios-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}], [{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links': 'https://example.com/wordpress-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/android-portfolio'}, {'links': 'https://example.com/android-tv-portfolio'}]]


In [36]:
job = json_res
job['skills']

['Attention to Detail',
 'Customer Service',
 'Database Management Systems',
 'Information Management',
 'Problem Solving',
 'Oral Communication']

In [38]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Salman, a business development executive at FamCodings. FamCodings is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of FamCodings 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase FamCodings's portfolio: {link_list}
        Remember you are Salman, BDE at FamCodings. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert IT Solutions for NASA's Information Management Needs

Dear Hiring Manager,

I came across the Information Technology Specialist (Direct Hire) role at NASA and was impressed by the organization's commitment to innovation and excellence. As a Business Development Executive at FamCodings, I believe our expertise in AI and software consulting can help NASA achieve its goals in information management and records management solutions.

Our team at FamCodings has extensive experience in developing and implementing IT solutions that improve operational efficiency, streamline processes, and enhance data management. We have successfully empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

In particular, our expertise in database management systems, cloud environments, and data architecture can help NASA establish and implement procedures to improve the effectiveness and efficiency 